<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loading-Data" data-toc-modified-id="Loading-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading Data</a></span></li><li><span><a href="#ImageDataGenerator" data-toc-modified-id="ImageDataGenerator-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>ImageDataGenerator</a></span><ul class="toc-item"><li><span><a href="#Preprocessing-images" data-toc-modified-id="Preprocessing-images-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Preprocessing images</a></span></li><li><span><a href="#define-testing,training-,-validation,resize,-chuck-data-and-scale" data-toc-modified-id="define-testing,training-,-validation,resize,-chuck-data-and-scale-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>define testing,training , validation,resize, chuck data and scale</a></span></li></ul></li><li><span><a href="#Base-Model" data-toc-modified-id="Base-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Base Model</a></span></li></ul></div>

In [4]:
# Importing Libraries
import numpy as np
import pandas as pd
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
from itertools import chain
from random import sample
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Lambda, Flatten,Activation, Dense, BatchNormalization, Conv2D
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input


###  Loading Data

In [6]:
train_data_dir =('/Users/khuloodnasher/dsc-mod-4-project-v2-1-onl01-dtsc-ft-030220/chest_xray/train')
test_data_dir =('/Users/khuloodnasher/dsc-mod-4-project-v2-1-onl01-dtsc-ft-030220/chest_xray/test')
val_data_dir =('/Users/khuloodnasher/dsc-mod-4-project-v2-1-onl01-dtsc-ft-030220/chest_xray/val')

### ImageDataGenerator
https://keras.io/api/preprocessing/image/

The images in the '/data' folder have various resolutions. We will reshape them so they are all 64 x 64 pixels.

#### Preprocessing images

#### define testing,training , validation,resize, chuck data and scale

In [11]:
# Get all the data in the directory data/test , and reshape them
print('test_size is:')
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(test_data_dir,
        target_size=(64, 64), batch_size=32,class_mode = 'binary')
print("==================================================================================")
# Get all the data in the directory data/train, and reshape them
print('train_size is:')
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(train_data_dir,
        target_size=(64, 64), batch_size=32,class_mode = 'binary')
print("==================================================================================")
print('val_size is:')

# Get all the data in the directory data/val, and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(val_data_dir,
        target_size=(64, 64), batch_size=32,class_mode = 'binary')

test_size is:
Found 624 images belonging to 2 classes.
train_size is:
Found 5216 images belonging to 2 classes.
val_size is:
Found 16 images belonging to 2 classes.


### Base Model